In [ ]:
import quantopian.pipeline.filters as Filters
import numpy as np
from datetime import datetime, timedelta
from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data import Fundamentals
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import CustomFactor, Returns, Latest, SimpleMovingAverage

    
    
MATERIALS_SECTOR = 101                                                                            
INDUSTRIALS_SECTOR = 310                                                                     
FINANCIALS_SECTOR = 103                                                               
UTILITIES_SECTOR = 207                                                                         
CONSUMER_DISCRETIONARY_SECTOR = 205                                                 
CONSUMER_STAPLES_SECTOR = 102                                                         
HEALTHCARE_SECTOR = 206                                                                       
TECHNOLOGY_SECTOR = 311                                                                       
ENERGY_SECTOR = 309                                                                               
COMMUNICATIONS_SECTOR = 308                                                       
PS_KEY = 'PS'                                                                                          
PE_KEY = 'PE'                                                                                          
PB_KEY = 'PB'                                                                                          
PFCF_KEY = 'PFCF'     
POCF_KEY = 'POCF'  

SECTOR_MEAN_RATIOS = {                                                                                        
  TECHNOLOGY_SECTOR: {                                                                                 
    PS_KEY: 2.5,                                                                                       
    PE_KEY: 29.9,                                                                                      
    PB_KEY: 7.62,                                                                                      
    PFCF_KEY: 1/.0481,                                                                                 
    POCF_KEY: 19.2,                                                                                    
  },                                                                                                   
  MATERIALS_SECTOR: {                                                                                  
    PS_KEY: 1,                                                                                         
    PE_KEY: 17,                                                                                        
    PB_KEY: 1.84,                                                                                      
    PFCF_KEY: 1/.0413,                                                                                 
    POCF_KEY: 10,                                                                                      
  },                                                                                                   
  INDUSTRIALS_SECTOR: {                                                                                
    PS_KEY: 1,                                                                                      
    PE_KEY: 19,                                                                                     
    PB_KEY: 5.03,                                                                                   
    PFCF_KEY: 1/.042,                                                                               
    POCF_KEY: 12,                                                                                   
  },                                                                                                
  FINANCIALS_SECTOR: {                                                                              
    PS_KEY: 3,                                                                                      
    PE_KEY: 13,                                                                                     
    PB_KEY: 1.62,                                                                                   
    PFCF_KEY: 10,                                                                                   
    POCF_KEY: 10,                                                                                   
  },    
  UTILITIES_SECTOR: {                                                                               
    PS_KEY: 2.9,                                                                                    
    PE_KEY: 25,                                                                                     
    PB_KEY: 2.19,                                                                                   
    PFCF_KEY: 10,                                                                                   
    POCF_KEY: 11.5,                                                                                 
  },                                                                                                
  CONSUMER_DISCRETIONARY_SECTOR: {                                                                  
    PS_KEY: .97,                                                                                    
    PE_KEY: 18,                                                                                     
    PB_KEY: 8.22,                                                                                   
    PFCF_KEY: 1/3.61,                                                                               
    POCF_KEY: 10.3,                                                                                 
  },                                                                                                
  CONSUMER_STAPLES_SECTOR: {                                                                        
    PS_KEY: 1.1,                                                                                    
    PE_KEY: 23,                                                                                     
    PB_KEY: 5.26,                                                                                   
    PFCF_KEY: 1/.0419,                                                                              
    POCF_KEY: 13.7,                                                                                 
  },                                                                                                
  ENERGY_SECTOR: {                                                                                  
    PS_KEY: 1.06,                                                                                   
    PE_KEY: 11.1,                                                                                   
    PB_KEY: 1.67,                                                                                   
    PFCF_KEY: 1/.0433,                                                                              
    POCF_KEY: 3.88,                                                                                 
  },                                                                                                
  COMMUNICATIONS_SECTOR: {                                                                          
    PS_KEY: 1.47,                                                                                   
    PE_KEY: 22,                                                                                     
    PB_KEY: 3.36,                                                                                         
    PFCF_KEY: 1/.0471,                                                                              
    POCF_KEY: 7.9,                                                                                  
  },
  HEALTHCARE_SECTOR: {                                                                          
    PS_KEY: 1,                                                                                   
    PE_KEY: 1,                                                                                     
    PB_KEY: 1,                                                                                         
    PFCF_KEY: 1,                                                                              
    POCF_KEY: 1,                                                                                  
  } 
}


class TTMFactor(CustomFactor):
    inputs = [
        Fundamentals.basic_eps_earnings_reports,
        Fundamentals.basic_eps_earnings_reports_asof_date,
        Fundamentals.fcf_per_share,
    ]
    window_length = 400
    window_safe = True
    outputs = ['eps_ttm', 'fcfps_ttm']

    def calc_ttm(self, values, dates):
        return [
            (v[d + np.timedelta64(52, 'W') > d[-1]])[
                np.unique(
                    d[d + np.timedelta64(52, 'W') > d[-1]],
                    return_index=True
                )[1]
            ].sum()
            for v, d in zip(values.T, dates.T)
        ]

    def compute(self, today, asset_ids, out, eps, eps_asof_date, fcfps):
        eps_ttm = self.calc_ttm(eps, eps_asof_date)
        fcfps_ttm = self.calc_ttm(fcfps, eps_asof_date)

        out.eps_ttm[:] = eps_ttm
        out.fcfps_ttm[:] = fcfps_ttm

    
    class CompositeFactor(CustomFactor):
        inputs = [
            Fundamentals.morningstar_sector_code,
            Fundamentals.sales_per_share, 
            Fundamentals.basic_eps_earnings_reports,
            Fundamentals.fcf_per_share,
            Fundamentals.book_value_per_share,
        ]
        window_length = 2
        outputs = ['sector_composite']
        
        def compute(self, today, asset_ids, out, sector, sps, eps, fcfps, bps):
            sector_composite = np.array([
                (
                    sps[0]#, 1) 
                    #SECTOR_MEAN_RATIOS.get(s, {}).get(PS_KEY, 1)
                    # vjw
                    #* max(eps[0], 1) * SECTOR_MEAN_RATIOS.get(s, {}).get(PE_KEY, 1)
                    #* max(fcfps[0], 1) * SECTOR_MEAN_RATIOS.get(s, {}).get(PFCF_KEY, 1)
                    #* max(bps[0], 1) * SECTOR_MEAN_RATIOS.get(s, {}).get(PB_KEY, 1)
                ) #** (1.0/4)
                for s in sector[0]
            ])
            out.sector_composite[:] = sector_composite
    

class EarningsFactor(CustomFactor):  
    # assign any default input(s). not required but maybe convenient.  

    # assign a default window_length. again not required  
    window_length = 2  
    # factors can have multiple outs. if so, then give them names  
    outputs = ['implied_earnings_growth', 
               'sector_mean_pe_price',]

    def compute(self, today, asset_ids, out, close, sector, eps_ttm):  
        # do any logic here  
        # the inputs appear in the same order as assigned above  
        # inputs are numpy arrays.  
        # columns are the assets. rows are the days with the earliest first [0] and most recent last [-1]  

        sector_mean_pe = np.array([SECTOR_MEAN_RATIOS.get(s, {}).get(PE_KEY, 1) for s in sector[0]])

        out.implied_earnings_growth[:] = 1.15 * (close[0]/eps_ttm[0]/sector_mean_pe)**.2 - 1
        out.sector_mean_pe_price[:] =  sector_mean_pe * eps_ttm[0]


# Gets the 52-week high for each asset.
class PriceFactor(CustomFactor):
    window_length = 252 # ~52 weeks
    inputs = [USEquityPricing.close]
    outputs = ['high52', 'low52']

    def compute(self, today, asset_ids, out, close):
        out.high52[:] = np.max(close, axis=0)
        out.low52[:] = np.min(close, axis=0)


class CompanyFactor(CustomFactor):
    window_length = 252 * 10  # ~10 years
    inputs = [
        Fundamentals.pe_ratio,
        Fundamentals.pb_ratio,
        Fundamentals.ps_ratio,
    ]
    outputs = [
        'max_pe', 'min_pe', 'max_pb', 'min_pb', 'max_ps', 'min_ps'
    ]
    def compute(self, today, asset_ids, out, pe, pb, ps):
        out.max_pe[:] = np.max(pe, axis=0)
        out.min_pe[:] = np.min(pe, axis=0)

        out.max_pb[:] = np.max(pb, axis=0)
        out.min_pb[:] = np.min(pb, axis=0)

        out.max_ps[:] = np.max(ps, axis=0)
        out.min_ps[:] = np.min(ps, axis=0)

def make_pipeline():
    # Base universe set to the QTradableStocksUS
    '''
    market_cap = Latest([Fundamentals.market_cap])
    market_cap_rank = market_cap.rank(mask=QTradableStocksUS())
    biggest = market_cap_rank.top(5)
    '''
    base_universe = Filters.StaticAssets(symbols([
        'AAPL',
        'GOOG',
        'MSFT',
        'AMZN',
        'NFLX',
        'TSLA',
        'V',
        'AXP',
        'JPM',
        'SQ',
        'PYPL',
        'SCHW',
        'DIS',
        'TXN',
        'T',
        'BRK_B',
    ]))

    cf = CompanyFactor()
    company_min_pe = cf.min_pe
    company_max_pe = cf.max_pe
    company_min_pb = cf.min_pb
    company_max_pb = cf.max_pb
    company_min_ps = cf.min_ps
    company_max_ps = cf.max_ps

    # Market
    pf = PriceFactor()
    high52 = pf.high52
    low52 = pf.low52

    # Factor of yesterday's close price.
    close = USEquityPricing.close.latest
    sps = Fundamentals.sales_per_share.latest
    eps = Fundamentals.basic_eps_earnings_reports.latest
    fcfps = Fundamentals.fcf_per_share.latest
    bps = Fundamentals.book_value_per_share.latest

    # Momentum
    ma50 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=50)
    ma200 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=200)

    #composite_factor = CompositeFactor()
    #sector_composite = composite_factor.sector_composite

    # Earnings
    ttmfactor = TTMFactor()
    ef = EarningsFactor(inputs=[
                    USEquityPricing.close,
                    Fundamentals.morningstar_sector_code,
                    ttmfactor.eps_ttm
                ])
    pe = Fundamentals.pe_ratio.latest

    company_mean_pe = SimpleMovingAverage(
        inputs = [Fundamentals.pe_ratio],
        window_length=365*5
    )
    implied_earnings_growth = ef.implied_earnings_growth
    sector = Fundamentals.morningstar_sector_code.latest
    eps_ttm = ttmfactor.eps_ttm

    company_mean_pe_price = eps_ttm*company_mean_pe
    sector_mean_pe_price = ef.sector_mean_pe_price
    market_pe_price = eps_ttm * 15

    company_min_pe_price = eps_ttm * company_min_pe
    company_max_pe_price = eps_ttm * company_max_pe

    # Book
    company_min_pb_price = Fundamentals.book_value_per_share.latest * company_min_pb
    company_max_pb_price = Fundamentals.book_value_per_share.latest * company_max_pb
    
    # Sales
    company_min_ps_price = Fundamentals.sales_per_share.latest * company_min_ps
    company_max_ps_price = Fundamentals.sales_per_share.latest * company_max_ps

    # Other
    fcf_10cap = ttmfactor.fcfps_ttm

    #lynch = eps_ttm * Fundamentals.diluted_eps_growth.latest

    pipe = Pipeline(
        columns={
            '  close': close,
            'sector' : sector,

            '_sps': sps,
            '_eps': eps,
            '_fcfps': fcfps,
            '_bps': bps,

            'high52': high52,
            'low52' : low52,

            'ma50'  : ma50,
            'ma200' : ma200,

            ' implied_earnings_growth': implied_earnings_growth,
            '$company_mean_pe_price': company_mean_pe_price,
            '$sector_mean_pe_price': sector_mean_pe_price,
            '$market_pe_price': market_pe_price,
            '$company_min_pe_price': company_min_pe_price,
            '$company_max_pe_price' : company_max_pe_price,

            '#company_min_pb_price': company_min_pb_price,
            '#company_max_pb_price' : company_max_pb_price,
            
            '^company_min_ps_price': company_min_ps_price,
            '^company_max_ps_price': company_max_ps_price,
            
            #'fcf_10cap' : fcf_10cap,
            #'sector_composite': sector_composite,
            #'lynch'    : lynch,
        },
        screen=base_universe
    )
    return pipe

today = datetime.now().date() - timedelta(days=1)

result = run_pipeline(make_pipeline(), today, today)

valuation_cols = [
    '^company_min_ps_price',
    '^company_max_ps_price',
    '$company_mean_pe_price',
    '$sector_mean_pe_price',
    '$market_pe_price',
    '$company_min_pe_price',
    '$company_max_pe_price',
    '#company_min_pb_price',
    '#company_max_pb_price',
    
    'high52',
    'low52',
    'ma50',
    'ma200',

    #'fcf_10cap',
    #'sector_composite',
]

result.insert(0, 'avg', result[valuation_cols].mean(1).round(2))
result
